In [ ]:
import os
import cv2
import json
import numpy as np
import threading
from skimage.color import rgb2lab, deltaE_cie76
from tqdm import tqdm

# Define the paths for keyframes and output
keyframes_path = '/path/to/keyframes'
output_file_path = '/path/to/output'
os.makedirs(output_file_path, exist_ok=True)

# Define a custom color palette
color_palette = {
    'black': [0, 0, 0],
    'blue': [0, 0, 255],
    'brown': [150, 75, 0],
    'green': [0, 128, 0],
    'grey': [128, 128, 128],
    'orange': [255, 165, 0],
    'pink': [255, 192, 203],
    'purple': [128, 0, 128],
    'red': [255, 0, 0],
    'white': [255, 255, 255],
    'yellow': [255, 255, 0]
}
palette_rgb = list(color_palette.values())
palette_lab = rgb2lab(np.uint8([palette_rgb])).reshape(-1, 3)
id2colors = list(color_palette.keys())

# Grid specifications
n_rows, n_cols = 7, 7  # Define grid size

def match_image_to_palette(segment, threshold=0.07):
    h, w, _ = segment.shape
    n_pixels = h * w
    segment_lab = rgb2lab(np.uint8(segment)).reshape(-1, 1, 3)
    diff = deltaE_cie76(segment_lab, palette_lab)
    color_indices, color_frequencies = np.unique(np.argsort(diff)[:, 0], return_counts=True)
    significant_colors = color_indices[color_frequencies > n_pixels * threshold]
    return [id2colors[index] for index in significant_colors]

def process_segment(x0, y0, x1, y1, image, result, idx):
    segment = image[y0:y1, x0:x1]
    result[idx] = match_image_to_palette(segment)

def process_image(image_path):
    image = cv2.imread(image_path)
    if image is None:
        return None
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    h, w, _ = image.shape
    segment_height = h // n_rows
    segment_width = w // n_cols

    results = [None] * (n_rows * n_cols)
    threads = []

    for i in range(n_rows):
        for j in range(n_cols):
            x0, y0 = j * segment_width, i * segment_height
            x1, y1 = (j + 1) * segment_width, (i + 1) * segment_height
            idx = i * n_cols + j
            thread = threading.Thread(target=process_segment, args=(x0, y0, x1, y1, image, results, idx))
            thread.start()
            threads.append(thread)

    for thread in threads:
        thread.join()

    return results

# Process each image in the directory
for filename in tqdm(os.listdir(keyframes_path)):
    if filename.endswith('.jpg'):
        file_path = os.path.join(keyframes_path, filename)
        dominant_colors = process_image(file_path)
        if dominant_colors:
            output_path = os.path.join(output_file_path, f'{os.path.splitext(filename)[0]}.json')
            with open(output_path, 'w') as file:
                json.dump(dominant_colors, file)
            print(f"Saved color extraction result for {filename} to {output_path}")
        else:
            print(f"Failed to process image {file_path}")
